# Gap analysis of US Sales and Use tax calendars


Install libraries from requirements.txt using pip

In [558]:
import sys
#!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install -r requirements.txt
print("requirements installed \n running program...")

  Using cached https://files.pythonhosted.org/packages/44/48/def306413b25c3d01753603b1a222a011b8621aed27cd7f89cbc27e6b0f4/xlwt-1.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3d/9e/b8a10bec874d2f6f2668b0d71ddf0d0a69ed9307d2ef111607adf81fdc1c/Pillow-5.2.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/5c/ee/e4acac810a85da614a60bf2221535bc2517d553b8d733cfd2dd644e2ab15/lxml-4.2.3-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/41/c6/78a9a0d0150dbf43095c6f422fdf6f948e18453c5ebbf92384175b372ca2/selenium-3.13.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a0/38/dcf83532480f25284f3ef13f8ed63e03c58a65c9d3ba2a6a894ed9497207/jdcal-1.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9e/d4/10f46e5cfac773e22707237bfcd51bbffeaf0a576b0a847ec7ab15bd7ace/beautifulsoup4-4.6.0-py3-none-any.whl
  Using cached https://files.pythonhoste

# Pandas

Data comes from xslx file containing US Sales and Use Tax deadlines and csv file for comparison. 

See AJAXwebscraping.ipynb for AJAX webscraping

We will use Pandas to visualize the data and to identify differences in tax deadlines


In [559]:
import pandas as pd
import re
import csv
import matplotlib as plt
import numpy as np

filename = "UScalendar.xlsx"
df = pd.read_excel(filename,header=2) # headers are on row 2 of excel spreadsheet
df.head()


,Due date,Unnamed: 1,Tax type,Unnamed: 3,Jurisdiction,Form,Unnamed: 6
0,2018-01-02 00:00:00,NaN,Sales & Use,NaN,North Dakota,ST,NaN
1,Monthly filers required to file Form ST for sales and use tax collected in November,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-02 00:00:00,NaN,Sales & Use,NaN,Wisconsin,ST-12,NaN
3,"Monthly filers owing $3,600 or less per quarter required to file Form ST-12 for sales and use tax collected in November",NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-02 00:00:00,NaN,Sales & Use,NaN,Indiana,ST-103,NaN


# DATA CLEAN UP
Formatting of spreadsheet is problematic. Heading is on row 2 and the even rows of the spreadsheet are a description of the preceding row. 

Delete unwanted columns "unnamed:1,3,6", make all lowercase and simplify column names

In [560]:
df.columns = [x.lower() for x in df.columns]
df = df.rename(columns={"due date":"date", "tax type":"tax"})
df.columns

Index(['date', 'unnamed: 1', 'tax', 'unnamed: 3', 'jurisdiction', 'form', 'unnamed: 6'], dtype='object')

In [561]:

del df['unnamed: 1']
del df['unnamed: 3']
del df['unnamed: 6']
df.head()


,date,tax,jurisdiction,form
0,2018-01-02 00:00:00,Sales & Use,North Dakota,ST
1,Monthly filers required to file Form ST for sales and use tax collected in November,NaN,NaN,NaN
2,2018-01-02 00:00:00,Sales & Use,Wisconsin,ST-12
3,"Monthly filers owing $3,600 or less per quarter required to file Form ST-12 for sales and use tax collected in November",NaN,NaN,NaN
4,2018-01-02 00:00:00,Sales & Use,Indiana,ST-103


Odd number rows contain the description for the even and 0 numbered rows.

In [562]:
df.tail()

,date,tax,jurisdiction,form
2159,Monthly filers required to file Form OS-114 for sales and use tax collected in November,NaN,NaN,NaN
2160,2018-12-31 00:00:00,Sales & Use,Nevada,TXR-01.01
2161,Monthly filers required to file Form TXR-01.01 for sales and use tax collected in November,NaN,NaN,NaN
2162,NaN,NaN,NaN,NaN
2163,©2018 CCH Incorporated and its affiliates and licensors. All rights reserved.,NaN,NaN,NaN


In [563]:
df.set_index("date") # set the date column as the index
df.dtypes
   

    

date            object
tax             object
jurisdiction    object
form            object
dtype: object

The comments are all strings whereas dates can be converted into datetime objects

In [564]:
c = df[df.date.str.contains("") == True] # extract the comments (strings) from date field 

In [565]:
df.head()

,date,tax,jurisdiction,form
0,2018-01-02 00:00:00,Sales & Use,North Dakota,ST
1,Monthly filers required to file Form ST for sales and use tax collected in November,NaN,NaN,NaN
2,2018-01-02 00:00:00,Sales & Use,Wisconsin,ST-12
3,"Monthly filers owing $3,600 or less per quarter required to file Form ST-12 for sales and use tax collected in November",NaN,NaN,NaN
4,2018-01-02 00:00:00,Sales & Use,Indiana,ST-103


In [567]:
df['date'] = pd.to_datetime(df['date'], errors='coerce') # turning dates into datetime object for conversion

In [568]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 4 columns):
date            1070 non-null datetime64[ns]
tax             1070 non-null object
jurisdiction    1070 non-null object
form            1070 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 67.7+ KB


In [569]:
df['date'] = df['date'].dt.strftime('%d/%m/%Y') # format is to match df_calendar imported later. Date format is UK dd/mm/yyyy

In [570]:
df.head()

,date,tax,jurisdiction,form
0,02/01/2018,Sales & Use,North Dakota,ST
1,NaT,NaN,NaN,NaN
2,02/01/2018,Sales & Use,Wisconsin,ST-12
3,NaT,NaN,NaN,NaN
4,02/01/2018,Sales & Use,Indiana,ST-103


In [571]:
df2 = df[df.form.notnull()] # remove rows which are null to clean up data. Using the column 'form'

In [573]:
df2[df2['jurisdiction'].str.contains("Louisiana")]



,date,tax,jurisdiction,form
100,22/01/2018,Sales & Use,Louisiana,R-1029
102,22/01/2018,Sales & Use,Louisiana,R-1029
374,20/02/2018,Sales & Use,Louisiana,R-1029
522,20/03/2018,Sales & Use,Louisiana,R-1029
682,20/04/2018,Sales & Use,Louisiana,R-1029
684,20/04/2018,Sales & Use,Louisiana,R-1029
890,21/05/2018,Sales & Use,Louisiana,R-1029
1032,20/06/2018,Sales & Use,Louisiana,R-1029
1204,20/07/2018,Sales & Use,Louisiana,R-1029
1206,20/07/2018,Sales & Use,Louisiana,R-1029


In [574]:
print(len(c)) # comparison on lengths of c and df2 dataframe
print(len(df2))
c.tail()

1082
1070


,date,tax,jurisdiction,form
2155,Monthly filers required to file Form TC-62 for sales and use tax collected in November,NaN,NaN,NaN
2157,Monthly filers required to file Form 41 for sales and use tax collected in November,NaN,NaN,NaN
2159,Monthly filers required to file Form OS-114 for sales and use tax collected in November,NaN,NaN,NaN
2161,Monthly filers required to file Form TXR-01.01 for sales and use tax collected in November,NaN,NaN,NaN
2163,©2018 CCH Incorporated and its affiliates and licensors. All rights reserved.,NaN,NaN,NaN


Length of comments and df2 don't match
Comments have paragraph headings for Month and Year interspersed among them

In [576]:
c[c['date'].str.contains("2018")] # all unwanted rows contain 2018

,date,tax,jurisdiction,form
325,February 2018,NaN,NaN,NaN
467,March 2018,NaN,NaN,NaN
595,April 2018,NaN,NaN,NaN
841,May 2018,NaN,NaN,NaN
981,June 2018,NaN,NaN,NaN
1111,July 2018,NaN,NaN,NaN
1373,August 2018,NaN,NaN,NaN
1515,September 2018,NaN,NaN,NaN
1637,October 2018,NaN,NaN,NaN
1885,November 2018,NaN,NaN,NaN


In [577]:
c2 = c[~c['date'].str.contains("2018")]
comments = c2['date']# removing 2018 rows from comments

In [578]:
df2 = df2.assign(comments=comments.values)



In [579]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1070 entries, 0 to 2160
Data columns (total 5 columns):
date            1070 non-null object
tax             1070 non-null object
jurisdiction    1070 non-null object
form            1070 non-null object
comments        1070 non-null object
dtypes: object(5)
memory usage: 50.2+ KB


In [613]:
df2.head()

,date,tax,jurisdiction,form,comments,id
26,22/01/2018,Sales & Use,Mississippi,72-010,Monthly filers required to file Form 72-010 for sales tax collected in December of previous year,US
28,22/01/2018,Sales & Use,Mississippi,72-110,Monthly filers required to file Form 72-110 for use tax collected in December of previous year,US
30,22/01/2018,Sales & Use,Mississippi,72-010,Quarterly filers required to file Form 72-010 for sales tax collected in the 4th quarter of previous year,US
32,22/01/2018,Sales & Use,Mississippi,72-110,Quarterly filers required to file Form 72-110 for use tax collected in the 4th quarter of previous year,US
34,22/01/2018,Sales & Use,New Jersey,ST-50,Monthly filers required to file quarterly return (Form ST-50) and remit sales and use tax collected in December of previous year,US


# Website

Load dates from csv file for comparison after AJAX webscraping

In [583]:
file = "2018-07-05CCHdates.csv"
parse_dates = ['date']
df_calendar = pd.read_csv(file,names=['date','country', 'comments'], parse_dates=parse_dates, dayfirst=True)
df_calendar.head()



,date,country,comments
0,2018-01-22,United States,Monthly sales and use tax for businesses in Alabama
1,2018-01-22,United States,Quarterly sales and use tax for businesses in Alabama
2,2018-01-22,United States,Annual sales and use tax for businesses in Alabama
3,2018-01-22,United States,Monthly estimated sales and use tax for businesses in Alabama
4,2018-02-20,United States,Monthly estimated sales and use tax for businesses in Alabama


In [584]:
df_calendar['date'] = df_calendar['date'].dt.strftime('%d/%m/%Y') # format date to match df2

In [585]:
df_calendar.head()

,date,country,comments
0,22/01/2018,United States,Monthly sales and use tax for businesses in Alabama
1,22/01/2018,United States,Quarterly sales and use tax for businesses in Alabama
2,22/01/2018,United States,Annual sales and use tax for businesses in Alabama
3,22/01/2018,United States,Monthly estimated sales and use tax for businesses in Alabama
4,20/02/2018,United States,Monthly estimated sales and use tax for businesses in Alabama


Date contains unwanted column "country" and the name of the State is in the comments. Need to extract the state name as jurisdiction column from the final word of each comment

In [586]:
df_calendar['comments'].head()

0     Monthly sales and use tax for businesses in Alabama          
1     Quarterly sales and use tax for businesses in Alabama        
2     Annual sales and use tax for businesses in Alabama           
3     Monthly estimated sales and use tax for businesses in Alabama
4     Monthly estimated sales and use tax for businesses in Alabama
Name: comments, dtype: object

In [587]:
juri = df_calendar['comments'].str.split(" in ") # Name of state appears after "businesses in"
# NB need to split on " in " and not "in" to avoid splitting states with "in" middle of name





In [588]:
juris = []
for i in range(len(juri)):
    juris.append(juri[i][-1])
print(juris)

['Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Alabama', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arizona', 'Arkansas', 'Arkansas', 'Arkansas', 'Arkansas', 'Arkansas', 'Arkansas', 'Arkansas', 'Arkansas', '

In [589]:
se = pd.Series(juris) # need to make list into a pd series before assigning

df_calendar = df_calendar.assign(jurisdiction=se.values)


In [590]:
df_calendar

,date,country,comments,jurisdiction
0,22/01/2018,United States,Monthly sales and use tax for businesses in Alabama,Alabama
1,22/01/2018,United States,Quarterly sales and use tax for businesses in Alabama,Alabama
2,22/01/2018,United States,Annual sales and use tax for businesses in Alabama,Alabama
3,22/01/2018,United States,Monthly estimated sales and use tax for businesses in Alabama,Alabama
4,20/02/2018,United States,Monthly estimated sales and use tax for businesses in Alabama,Alabama
5,20/02/2018,United States,Monthly sales and use tax for businesses in Alabama,Alabama
6,20/03/2018,United States,Monthly estimated sales and use tax for businesses in Alabama,Alabama
7,20/03/2018,United States,Monthly sales and use tax for businesses in Alabama,Alabama
8,20/04/2018,United States,Monthly estimated sales and use tax for businesses in Alabama,Alabama
9,20/04/2018,United States,Monthly sales and use tax for businesses in Alabama,Alabama


# Comparison
Purpose of data comparison is to determine whether any dates are missing/different in df_calendar compared to df2

df_calendar is ordered by states but df2 is ordered by date

In [593]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061 entries, 0 to 1060
Data columns (total 4 columns):
date            1061 non-null object
country         1061 non-null object
comments        1061 non-null object
jurisdiction    1061 non-null object
dtypes: object(4)
memory usage: 33.2+ KB


In [594]:
df_calendar.sort_values("date")

,date,country,comments,jurisdiction
1036,01/10/2018,United States,Monthly Sales and Use Tax filing for businesses in Wisconsin,Wisconsin
1056,01/10/2018,United States,Monthly Sales and Use Tax filing for businesses in Wyoming,Wyoming
258,01/10/2018,United States,Monthly sales and use tax for businesses in Indiana,Indiana
298,01/10/2018,United States,Monthly filers required to file Form 32-022 for sales and use tax for businesses in Iowa,Iowa
692,01/10/2018,United States,Monthly Sales and Use Tax filing for businesses in North Dakota,North Dakota
111,01/10/2018,United States,Monthly sales and use tax for businesses in California,California
609,01/10/2018,United States,Monthly Sales and Use Tax filing for businesses in Nevada,Nevada
145,01/10/2018,United States,Monthly sales and use tax for businesses in Connecticut,Connecticut
927,01/10/2018,United States,Monthly Sales and Use Tax filing for businesses in Utah,Utah
137,02/04/2018,United States,Monthly sales and use tax for businesses in Connecticut,Connecticut


In [595]:
df_calendar.set_index("date")
df_calendar.sort_values("date")

,date,country,comments,jurisdiction
1036,01/10/2018,United States,Monthly Sales and Use Tax filing for businesses in Wisconsin,Wisconsin
1056,01/10/2018,United States,Monthly Sales and Use Tax filing for businesses in Wyoming,Wyoming
258,01/10/2018,United States,Monthly sales and use tax for businesses in Indiana,Indiana
298,01/10/2018,United States,Monthly filers required to file Form 32-022 for sales and use tax for businesses in Iowa,Iowa
692,01/10/2018,United States,Monthly Sales and Use Tax filing for businesses in North Dakota,North Dakota
111,01/10/2018,United States,Monthly sales and use tax for businesses in California,California
609,01/10/2018,United States,Monthly Sales and Use Tax filing for businesses in Nevada,Nevada
145,01/10/2018,United States,Monthly sales and use tax for businesses in Connecticut,Connecticut
927,01/10/2018,United States,Monthly Sales and Use Tax filing for businesses in Utah,Utah
137,02/04/2018,United States,Monthly sales and use tax for businesses in Connecticut,Connecticut


In [614]:
df2= df2[13:] # strip out first 13 results from df2 so it starts on same date as df_calendar

## Finding all differences between DF2 and DF_Calendar using Python Dict

In [615]:
map_dict = {} # dictionary to record whether matching date and state found

# iterate each row of df2 against df_calendar
for index, row in df2.iterrows():
    dict_k = str(row['date'])
    dict_key = dict_k + " " + row['jurisdiction'] # ID needs to be date plus jurisdiction
    map_dict[dict_key] = "unmatched" 
    for index2,row2 in df_calendar.iterrows():
        if row['date'] == row2['date']:
            if row['jurisdiction'].lower() == row2['jurisdiction'].lower():
                map_dict[dict_key] = "matched"
   

In [601]:
# print all dates that did not match date and state in df_calendar
for k,v in map_dict.items():
    if v == "unmatched":
        print(k,v)
#        

30/03/2018 Indiana unmatched
22/10/2018 New York unmatched
02/03/2018 Indiana unmatched
20/08/2018 District of Columbia unmatched
NaT Jurisdiction unmatched
20/11/2018 District of Columbia unmatched
29/06/2018 Arizona unmatched
20/04/2018 District of Columbia unmatched
20/03/2018 District of Columbia unmatched
20/07/2018 District of Columbia unmatched
20/02/2018 District of Columbia unmatched
20/09/2018 District of Columbia unmatched
22/01/2018 District of Columbia unmatched
22/10/2018 District of Columbia unmatched
20/06/2018 District of Columbia unmatched
21/05/2018 District of Columbia unmatched
20/12/2018 District of Columbia unmatched
23/05/2018 South Dakota unmatched


In [616]:
pd.set_option('display.max_colwidth', 1) # increase width of dfs to make easier to see

## Finding differences in calendars using Pandas DataFrames

In [604]:
df2= df2.assign(id="US") # assign ids to dfs to make it easier to compare in merged df

In [606]:
df_calendar = df_calendar.assign(id="CCH")

In [617]:
df_merged = df2.merge(df_calendar, how='outer') # merge dataframes with outer join
df_merged = df_merged[['date', 'jurisdiction', 'comments', 'id']] # drop unwanted columns

In [619]:
df_merged.head()

,date,jurisdiction,comments,id
0,22/01/2018,Tennessee,Annual filers required to file Form SLS 450 for sales and use tax collected in previous year,US
1,22/01/2018,Texas,Monthly filers required to file Form 01-114 for sales and use tax collected in December of previous year,US
2,22/01/2018,Texas,Quarterly filers required to file Form 01-114 for sales and use tax collected in the 4th quarter of previous year,US
3,22/01/2018,Texas,Annual filers required to file Form 01-114 for sales and use tax collected in previous year,US
4,22/01/2018,Virginia,Monthly filers required to file Form ST-9 for sales tax collected in December of previous year,US


Find duplicate date and jurisdictions with boolean mask

In [609]:
duplicate_boolean = df_merged.duplicated(['date','jurisdiction'],keep=False) # Need keep=False shows all duplicates
unique = df_merged.loc[duplicate_boolean == False] # unique values don't have a duplicate

In [610]:
unique.sort_values('jurisdiction')

,date,jurisdiction,comments,id
542,29/06/2018,Arizona,"Taxpayers with liability in excess of $1,000,000 in previous calendar year required to file paper Form TPT-ES for estimated tax due",US
20,22/01/2018,District of Columbia,Monthly filers required to file Form FR-800M for sales and use tax collected in December of previous year,US
1006,20/12/2018,District of Columbia,Monthly filers required to file Form FR-800M for sales and use tax collected in November,US
938,20/11/2018,District of Columbia,Monthly filers required to file Form FR-800M for sales and use tax collected in October,US
758,20/09/2018,District of Columbia,Monthly filers required to file Form FR-800M for sales and use tax collected in August,US
689,20/08/2018,District of Columbia,Monthly filers required to file Form FR-800M for sales and use tax collected in July,US
574,20/07/2018,District of Columbia,Monthly filers required to file Form FR-800M for sales and use tax collected in June,US
497,20/06/2018,District of Columbia,Monthly filers required to file Form FR-800M for sales and use tax collected in May,US
428,21/05/2018,District of Columbia,Monthly filers required to file Form FR-800M for sales and use tax collected in April,US
320,20/04/2018,District of Columbia,Monthly filers required to file Form FR-800M for sales and use tax collected in March,US
